In [2]:
import os
import pandas as pd
from rouge_score import rouge_scorer
import sacrebleu
from bert_score import score

Tính điểm BLEU, ROUGE và BERT

In [3]:
# Đường dẫn chứa file CSV kết quả
results_dir = "D:\\NLP\\results"  # Thay bằng thư mục chứa file kết quả
labels_dir = "D:\\NLP\\NLP_Labels"  # Thư mục chứa file nhãn gốc

# Hàm load nhãn gốc
def load_labels():
    labels = {}
    for i in range(1, 201):
        label_file = os.path.join(labels_dir, f"label-{i}.md")
        with open(label_file, "r", encoding="utf-8") as f:
            labels[f"output_{i}.json"] = f.read().strip()
    return labels

# Hàm tính ROUGE
def compute_rouge(predicted, reference):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, predicted)
    return {key: value.fmeasure for key, value in scores.items()}

# Hàm tính BLEU
def compute_bleu(predicted, reference):
    bleu = sacrebleu.sentence_bleu(predicted, [reference])
    return bleu.score

# Hàm tính BERTScore
def compute_bertscore(predicted_list, reference_list):
    P, R, F1 = score(predicted_list, reference_list, lang="vi", verbose=True)
    return F1.mean().item()

# Chạy đánh giá
def main():
    labels = load_labels()  # Load nhãn gốc
    results = []

    # Duyệt qua từng file kết quả của các mô hình
    for result_file in os.listdir(results_dir):
        if result_file.endswith(".csv"):
            # Phân loại mô hình và phương pháp học
            if "zeroshot" in result_file.lower():
                learning_type = "zeroshot"
            else:
                learning_type = "fewshot"
            model_name = result_file.replace(".csv", "").replace("_zero", "").replace("_few", "")
            print(f"Đang đánh giá mô hình: {model_name}, loại học: {learning_type}")

            # Đọc file kết quả
            result_path = os.path.join(results_dir, result_file)
            df = pd.read_csv(result_path)
            
            # Lưu kết quả đánh giá
            model_scores = {"model": model_name, "learning_type": learning_type, "rouge1": [], "rouge2": [], "rougeL": [], "bleu": [], "bertscore": []}

            predicted_list = []
            reference_list = []

            for _, row in df.iterrows():
                file_name = row["file_name"]
                predicted = row["predicted_label"]
                reference = labels[file_name]

                # Tính ROUGE
                rouge_scores = compute_rouge(predicted, reference)
                model_scores["rouge1"].append(rouge_scores["rouge1"])
                model_scores["rouge2"].append(rouge_scores["rouge2"])
                model_scores["rougeL"].append(rouge_scores["rougeL"])

                # Tính BLEU
                bleu_score = compute_bleu(predicted, reference)
                model_scores["bleu"].append(bleu_score)

                # Thu thập dữ liệu cho BERTScore
                predicted_list.append(predicted)
                reference_list.append(reference)

            # Tính BERTScore
            bertscore = compute_bertscore(predicted_list, reference_list)
            model_scores["bertscore"] = [bertscore] * len(predicted_list)  # Gán giá trị chung cho từng hàng

            # Tính trung bình của các độ đo
            results.append({
                "model": model_name,
                "learning_type": learning_type,
                "avg_rouge1": sum(model_scores["rouge1"]) / len(model_scores["rouge1"]),
                "avg_rouge2": sum(model_scores["rouge2"]) / len(model_scores["rouge2"]),
                "avg_rougeL": sum(model_scores["rougeL"]) / len(model_scores["rougeL"]),
                "avg_bleu": sum(model_scores["bleu"]) / len(model_scores["bleu"]),
                "avg_bertscore": model_scores["bertscore"][0],  # Giá trị đã tính chung
            })

    # Lưu kết quả đánh giá tổng hợp
    results_df = pd.DataFrame(results)
    results_df.to_csv("D:\\NLP\\results\\evaluation_results.csv", index=False, encoding="utf-8")
    print("Kết quả đánh giá đã được lưu vào evaluation_results.csv")
    return results

In [4]:
# Chạy chương trình
if __name__ == "__main__":
    results = main()

Đang đánh giá mô hình: meta-llama_Llama-3.3-70B-Instruct-Turbo-fewshot, loại học: fewshot
calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 22.48 seconds, 8.81 sentences/sec
Đang đánh giá mô hình: meta-llama_Llama-3.3-70B-Instruct-Turbo-zeroshot, loại học: zeroshot
calculating scores...
computing bert embedding.


...

done in 26.48 seconds, 7.55 sentences/sec
Đang đánh giá mô hình: mistralai_Mixtral-8x22B-Instruct-v0.1-zeroshot, loại học: zeroshot
calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 22.94 seconds, 8.63 sentences/sec
Kết quả đánh giá đã được lưu vào evaluation_results.csv
